In [5]:
# IMPORT LIBRARIES

import numpy as np 
import sympy as sp
import matplotlib.pyplot as plt

In [6]:
# DATA 

D_Core = 0.7 # Diffusion coefficient (tank) [cm]
D_Person = 0.16 # Diffusion coefficient (person) [cm]
h = 40 # Tank height [cm]
R = 20 # Tank / person radius [cm]
rho_H2O = 1 # Water density [g/cm3]
NAV = 6.022 * 10 ** 23 # Avogadro number [atoms/mol]
MM_H2O = 18.015 # Water molar mass [g/mol]
nu_0 = 2.405 # Fundamental solution for the radial part (Bessel)

In [7]:
# FUNCTION AND SYMBOLIC VARIABLES DEFINITION

z = sp.symbols('z')
r = sp.symbols('r')
lam = sp.symbols('lambda', nonnegative = True) 
alfa = sp.symbols('alfa', nonnegative = True)
L_c = sp.symbols('L_c', nonnegative = True)
L_p = sp.symbols('L_p', nonnegative = True)
D_c = sp.symbols('D_c', nonnegative = True) 
D_p = sp.symbols('D_p', nonnegative = True) 
H = sp.symbols('H', nonnegative = True) # Cylinder height
h = sp.symbols('h', nonnegative = True) # Person height
F_cyl_z = sp.Function('phi_c,z', real = True)(z)
F_person_z = sp.Function('phi_p,z', real = True)(z)
F_cyl_radial = sp.Function('phi_c,r', real = True)(r) # Non so se serva, in realtà la parte radiale serve solo per B_g^2 = B_z^2 + B_r^2, con B_r^2 = (nu_0 / R)^2

In [8]:
# ODE + BC

ODE_c_z = sp.Eq(sp.diff(sp.diff(F_cyl_z, z)) + lam** 2 * F_cyl_z, 0)
print('AXIAL ODE FOR THE CORE:')
display(ODE_c_z)

# ODE_c_r = sp.Eq((1 /r) * sp.diff(1/r *sp.diff(F_cyl_radial, r)) + alfa** 2 * F_cyl_z, 0)
# print('RADIAL ODE FOR THE CORE:')
# display(ODE_c_r)

ODE_p_z = sp.Eq(sp.diff(sp.diff(F_person_z, z)) - (1 / L_p** 2) * F_person_z, 0)
print('AXIAL ODE FOR THE PERSON:')
display(ODE_p_z)

flux_c_z = sp.dsolve(ODE_c_z)
sC1 = sp.symbols('C1')   
sC2 = sp.symbols('C2')
print('GENERAL SOLUTION FOR THE AXIAL FLUX IN THE CORE:')
display(flux_c_z)

flux_p_z = sp.dsolve(ODE_p_z)
sC3 = sp.symbols('C3')
sC4 = sp.symbols('C4')
flux_p_z = sp.Eq(
    flux_p_z.lhs,
    sC3 * sp.sinh(z/L_p) + sC4 * sp.cosh(z/L_p)
)
print('GENERAL SOLUTION FOR THE AXIAL FLUX IN THE PERSON:')
display(flux_p_z)

# flux_r = sp.dsolve(ODE_c_r)
# print('GENERAL SOLUTION FOR THE FLUX IN THE REFLECTOR:')
# display(flux_r)

# Symmetry BC
symmetry = sp.Eq(
    sp.diff(flux_c_z.rhs, z).subs(z, 0), 
    0 
)
print('SYMMETRY CONDITION IN THE CORE:')
display(symmetry)

# Continuity BC
continuity_f = sp.Eq(
    flux_c_z.rhs.subs(z, H/2), 
    flux_p_z.rhs.subs(z, H/2)
)
print('CONTINUITY CONDITION FOR THE FLUX AT THE INTERFACE:')
display(continuity_f)
continuity_curr = sp.Eq(
    -D_c* sp.diff(flux_c_z.rhs, z).subs(z, H/2),
    -D_p * sp.diff(flux_p_z.rhs, z).subs(z, H/2)
    )
print('CONTINUITY CONDITION FOR THE CURRENT AT THE INTERFACE:')
display(continuity_curr)

# Vacuum BC
vacuum_core = sp.Eq(
    flux_c_z.rhs.subs(z, -H/2),
    0
)
print('VACUUM CONDITION IN THE CORE:')
display(vacuum_core)
vacuum_person = sp.Eq(
    flux_p_z.rhs.subs(z, H/2+ h),
    0
)
print('VACUUM CONDITION IN THE PERSON:')
display(vacuum_person)

AXIAL ODE FOR THE CORE:


Eq(lambda**2*phi_c,z(z) + Derivative(phi_c,z(z), (z, 2)), 0)

AXIAL ODE FOR THE PERSON:


Eq(Derivative(phi_p,z(z), (z, 2)) - phi_p,z(z)/L_p**2, 0)

GENERAL SOLUTION FOR THE AXIAL FLUX IN THE CORE:


...

Eq(-D_c*(C1*lambda*cos(H*lambda/2) - C2*lambda*sin(H*lambda/2)), -D_p*(C3*cosh(H/(2*L_p))/L_p + C4*sinh(H/(2*L_p))/L_p))

VACUUM CONDITION IN THE CORE:


Eq(-C1*sin(H*lambda/2) + C2*cos(H*lambda/2), 0)

VACUUM CONDITION IN THE PERSON:


Eq(C3*sinh((H/2 + h)/L_p) + C4*cosh((H/2 + h)/L_p), 0)